In [127]:
import pandas as pd

In [123]:
def add_light_work(data):
    """
    Функция добавляет данные о времени включения света из файла 'time_city_light.csv'
    сырой датафрем подаем на вход
    """
    # обработаем файл с динамикой ВВП
    light = pd.read_csv('data/time_city_light.csv')
    light['turn_on'] = pd.to_datetime(light['on'], format='%H-%M').dt.time
    light['turn_off'] = pd.to_datetime(light['off'], format='%H-%M').dt.time
    light.drop(['on','off'], inplace=True, axis=1) 
        
    # обработаем основной фрейм - создадим столбец времени, который потом удалим
    data['date_temp'] = pd.to_datetime(data['date'], format = '%Y-%m-%d' )
    data['date_temp'] = data['date_temp'] + pd.to_timedelta(data['time'] , 'H')

    def is_light(input_dt):
        """
        Функция сравнивает значения с временем вкл. света.
        если освещение включено, возвращает 1. 
        """
        # извлекаем месяц и день из входящей даты
        month = input_dt.month
        day = input_dt.day
        # находим график работы освещения в справочном файле
        row = light.loc[(month==light['month'])  & (light['day']<=day),:].tail(1)

        # извлекаем время из timestamp и сравниваем 
        tm = input_dt.time()
        if tm<row.turn_off.values or tm>row.turn_on.values:
            return 1
        else:
            return 0
    
        
    data['is_light'] = data['date_temp'].map(is_light)
    data.drop('date_temp',axis=1,inplace=True)
       
    return data